In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rd

cols = ['Underlying']
strikeRange = [455,456,457,458,459,460,461]

for i in range(len(strikeRange)):
    cols += [f'Strike: ${strikeRange[i]}', f'call_openint {strikeRange[i]}', f'call_volume {strikeRange[i]}', f'call_bid {strikeRange[i]}',f'call_ask {strikeRange[i]}',f'put_bid {strikeRange[i]}',f'put_ask {strikeRange[i]}',f'put_volume {strikeRange[i]}',f'put_openint {strikeRange[i]}']

df = pd.DataFrame(columns=cols)

num_trades = 1500
for t in range(num_trades):
    if t > 0:
        df.loc[t,'Underlying'] = df.loc[t-1,'Underlying'] + np.random.normal(0,5)
    else:
        df.loc[t,'Underlying'] = 458
        for strike in range(len(strikeRange)):
            df.loc[t,f'call_openint {strikeRange[strike]}'] = round(np.random.normal(100,30))
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(400 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            
        
    for strike in range(len(strikeRange)):
        df.loc[t,f'Strike: ${strikeRange[strike]}'] = f'Strike: ${strikeRange[strike]}'

        #set volume and open interest
        if t>0:
            df.loc[t,f'call_openint {strikeRange[strike]}'] = max(df.loc[t-1,f'call_openint {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            # if OI jumps and vol doesn't, market responds heavily
            # if OI jumps and vol jumps, market responds medium
            # if OI doesn't jump and vol does, market responds medium
            # if neither jump, status quo
            old_cOI = df.loc[t-1,f'call_openint {strikeRange[strike]}']
            old_pOI = df.loc[t-1,f'put_openint {strikeRange[strike]}']
            old_cVol = df.loc[t-1,f'call_volume {strikeRange[strike]}']
            old_pVol = df.loc[t-1,f'put_volume {strikeRange[strike]}']

            cOI = df.loc[t,f'call_openint {strikeRange[strike]}']
            pOI = df.loc[t,f'put_openint {strikeRange[strike]}']
            cVol = df.loc[t,f'call_volume {strikeRange[strike]}']
            pVol = df.loc[t,f'put_volume {strikeRange[strike]}']

            # if cOI > old_cOI + 5 and cVol < old_cVol + 5:
            #     # call jumps, put drops
            # elif pOI > old_pOI + 5 and pVol < old_pVol + 5:
            #     # call drops, put jumps
            # elif c

            
            #get call data
            call_bid = np.random.normal(2,1)
            call_ask = call_bid-np.random.normal(0.2,0.1) #
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            #calculate put call parity to back out put bid/ask
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

            #get put data
            put_bid = mean_put+np.random.normal(0.2,0.1)
            put_ask = mean_put-np.random.normal(0.2,0.1)
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask
        
        else: #initial conditions
            #get call data
            call_bid = np.random.normal(2,1)
            call_ask = call_bid-np.random.normal(0.2,0.1)
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            #calculate put call parity to back out put bid/ask
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

            #get put data
            put_bid = mean_put+np.random.normal(0.2,0.1)
            put_ask = mean_put-np.random.normal(0.2,0.1)
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask

        

display(df)
df.to_csv('OptionsData1.csv')

,Underlying,Strike: $455,call_openint 455,call_volume 455,call_bid 455,call_ask 455,put_bid 455,put_ask 455,put_volume 455,put_openint 455,...,put_openint 460,Strike: $461,call_openint 461,call_volume 461,call_bid 461,call_ask 461,put_bid 461,put_ask 461,put_volume 461,put_openint 461
0,458,Strike: $455,97,103,1.236254,1.166805,-1.674958,-1.913985,0,303,...,286,Strike: $461,101,99,3.336746,3.265963,6.525156,6.133051,0,299
1,454.223648,Strike: $455,97,53,1.777722,1.550858,2.810252,2.096474,47,103,...,56,Strike: $461,102,48,2.868206,2.662053,9.758873,9.327369,52,98
2,453.889666,Strike: $455,107,43,0.380465,0.147077,1.71342,0.99134,57,93,...,56,Strike: $461,94,56,2.331859,2.178164,9.582463,9.234893,44,106
3,463.16783,Strike: $455,110,40,1.472963,1.302289,-6.672383,-7.015978,60,90,...,51,Strike: $461,102,48,3.954993,3.729792,1.724324,1.342106,52,98
4,464.041827,Strike: $455,126,24,2.68648,2.393658,-6.263316,-6.805521,76,74,...,59,Strike: $461,96,54,1.717815,1.577712,-1.067513,-1.482211,46,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,330.845037,Strike: $455,352,0,1.960343,1.759529,126.355202,125.735871,150,0,...,0,Strike: $461,104,46,2.58399,2.429853,132.802325,132.442956,54,96
1496,332.33166,Strike: $455,329,0,1.542174,1.356235,124.143087,123.934818,150,0,...,0,Strike: $461,113,37,1.402667,1.181189,130.101421,129.60722,63,87
1497,333.3566,Strike: $455,325,0,0.148203,0.016636,122.078717,121.53413,150,0,...,0,Strike: $461,113,37,1.154608,1.071475,129.05378,128.597998,63,87
1498,332.69895,Strike: $455,321,0,1.744349,1.547113,124.210438,123.755107,150,0,...,0,Strike: $461,104,46,1.488141,1.151073,129.88387,129.278822,54,96
